**Brainstorming**:
- comparison (soda generator)
- arithmetic
- bitshifts
- Note on semantics
- Exercises/examples: grammar, dice arithmetic, graph and symbolic computation, shortcuts
- formating?



Operator overloading
====================



In [13]:
from __future__ import annotations

from typing import Union
import random


class Gram:
    def __or__(self, other: Union[Gram, str]) -> OrRule:
        return OrRule(self, other)
    
    def __add__(self, other: Union[Gram, str]) -> CFRule:
        return CFRule(self, other)

class Rule(Gram):
    def __init__(self, *operands: Union[Gram, str]) -> None:
        super().__init__()
        def cast(x: Union[Gram, str]) -> Gram:
            if isinstance(x, str):
                x = Terminal(x)
            return x
        
        self._operands = tuple(cast(x) for x in operands)

class OrRule(Rule):
    def __str__(self) -> str:
        return str(random.choice(self._operands))
    
class CFRule(Rule):

    def __str__(self) -> str:
        return " ".join(str(x) for x in self._operands)
    


class Terminal(Gram):
    def __init__(self, symbol: str) -> None:
        self._symbol = symbol

    def __str__(self) -> str:
        return self._symbol
    
    
    


protagonist = (
    Terminal("Harry Potter") | "Luke Skywalker" | "Frodo Baggins"
)


actions = Terminal("must find") | "must destroy"

macguffin = (
    Terminal("the horcruxes") | "the death star" | "the one true ring"
)

victory = Terminal("to defeat") | "to beat" | "to vanquished" | "to rid the world of"

enemy = Terminal("Voldemort") | "the empire" | "Sauron"


story = protagonist + actions + macguffin + victory + enemy

for _ in range(5):
    print(story)


Harry Potter must find the one true ring to rid the world of the empire
Frodo Baggins must destroy the one true ring to beat Sauron
Luke Skywalker must find the death star to rid the world of Sauron
Frodo Baggins must find the horcruxes to beat the empire
Frodo Baggins must destroy the death star to rid the world of the empire
